In [4]:
import pandas as pd
import re
import konlpy
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
from tqdm import tqdm
import urllib.request

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-2, 2))

In [5]:
# 데이터 불러오기
df = pd.read_table('유튜브.csv')
df = df.rename(columns={"제목,본문,댓글": "sentence"})
df

,sentence
0,"윤석열 친이 노선 타면서 디시에서 개빨림,디시 일베가 윤석열만이 우리나라유일의 희..."
1,"어제자 그알 보고 느낀 점,코로나가 염병이긴 염병이구나코로나로 온 동네 노인정 다..."
2,"그알이 남긴 역사로 남을 짤,,조선일보도 못믿는다더라 탄핵시국에 통수쳤다고 촌의 뒤..."
3,"그알이 비겁한게 언레기는 안 다루더라,언레기 책임은 안 다루고 전부 유튜버로 몰아가..."
4,"한국만 예외인 고학력자의 진보화, 저학력 저소득층을 지지기반으로 가지던 좌파정당이..."
5,"그알에서 뭐랬길래 인정하는 사람들이 생김,남유게인지 여기인지 모르겠는데 반응 보니깐..."
6,"암만봐도 한강에 과몰입한건 딱 그 부모 정도 나이임,젊은층도 있겠지만 박종철 열사 ..."
7,"한강 사건이 말해주는 대형렉카와 좋소렉카의 차이,좋소렉카 어떻게든 끌고 갈려고 함..."
8,"한강사건 요약, 언론 믿지마 일베를믿어,지능이 딸리는것들이 유튜브 광신하더라대표적인..."
9,"오늘자 그알에 나온 유튜브 채널 곳,밀쳐지는 손을 봤다는 분수술용 메스를 휘둘렀다는..."


In [6]:
sent_dic = pd.read_csv('SentiWord_Dict.txt',sep = '\t',header=None)
sent_dic.iloc[14850,0]='갈등'

pos_dic = sent_dic[sent_dic[1]>0]
neg_dic = sent_dic[sent_dic[1]<0]
neu_dic = sent_dic[sent_dic[1]==0]

In [7]:
okt = konlpy.tag.Okt()

def text_preprocess(x):
    text=[]
    a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

def tokenize(x):
    text = []
    tokens = okt.pos(x)
    for token in tokens :
        if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
            text.append(token[0])
    return text

In [8]:
tqdm.pandas()
df['comment'] = df['sentence'].apply(lambda x : text_preprocess(x))
df['comment'] = df['comment'].progress_apply(lambda x: tokenize(x))
df

100%|██████████| 15/15 [00:08<00:00,  1.81it/s]


,sentence,comment
0,"윤석열 친이 노선 타면서 디시에서 개빨림,디시 일베가 윤석열만이 우리나라유일의 희...","[윤석열, 친, 이, 노선, 타면서, 디시, 에서, 개빨림, 디시, 일베, 가, 윤..."
1,"어제자 그알 보고 느낀 점,코로나가 염병이긴 염병이구나코로나로 온 동네 노인정 다...","[어제, 자, 그, 알, 보고, 느낀, 점, 코로나, 가, 염병, 이긴, 염병, 이..."
2,"그알이 남긴 역사로 남을 짤,,조선일보도 못믿는다더라 탄핵시국에 통수쳤다고 촌의 뒤...","[그, 알, 이, 남긴, 역사, 로, 남, 을, 짤, 조선일보, 도, 못, 믿는다더..."
3,"그알이 비겁한게 언레기는 안 다루더라,언레기 책임은 안 다루고 전부 유튜버로 몰아가...","[그, 알, 이, 비겁한게, 언레기, 는, 안, 다루더라, 언, 레기, 책임, 은,..."
4,"한국만 예외인 고학력자의 진보화, 저학력 저소득층을 지지기반으로 가지던 좌파정당이...","[한국, 만, 예외, 인, 고학력자, 의, 진보, 화, 저, 학력, 저, 소득, 층..."
5,"그알에서 뭐랬길래 인정하는 사람들이 생김,남유게인지 여기인지 모르겠는데 반응 보니깐...","[그, 알, 에서, 뭐랬길, 래, 인정, 하는, 사람, 들, 이, 생김, 남유, 게..."
6,"암만봐도 한강에 과몰입한건 딱 그 부모 정도 나이임,젊은층도 있겠지만 박종철 열사 ...","[암만, 봐도, 한강, 에, 과, 몰입, 한, 건, 딱, 그, 부모, 정도, 나이,..."
7,"한강 사건이 말해주는 대형렉카와 좋소렉카의 차이,좋소렉카 어떻게든 끌고 갈려고 함...","[한강, 사건, 이, 말, 해주는, 대형, 렉카, 와, 좋소, 렉카, 의, 차이, ..."
8,"한강사건 요약, 언론 믿지마 일베를믿어,지능이 딸리는것들이 유튜브 광신하더라대표적인...","[한강, 사건, 요약, 언론, 믿지마, 일베, 를, 믿어지, 능이, 딸리는것, 들이..."
9,"오늘자 그알에 나온 유튜브 채널 곳,밀쳐지는 손을 봤다는 분수술용 메스를 휘둘렀다는...","[오늘, 자, 그, 알, 에, 나온, 유튜브, 채널, 곳, 밀쳐지는, 손, 을, 봤..."


In [9]:
# 단어별 감성점수 측정

def make_sent_dict(x) :
    pos=[]
    neg=[]
    tmp={}
    
    for sentence in tqdm(x):
        for word in sentence :
            target = sent_dic[sent_dic[0]==word]
            if len(target)==1: # 기존에 있는 단어라면 그대로 사용
                score = float(target[1])
                if score > 0:
                    pos.append(word)
                elif score < 0:
                    neg.append(word)                
            tmp[word] = {'W':0,'WP':0,'WN':0} # 감성사전 구성
    pos = list(set(pos))
    neg = list(set(neg))
    
    for sentence in tqdm(x):
        for word in sentence :
            tmp[word]['W'] += 1 # 빈도 수
            for po in pos :
                if po in sentence:
                    tmp[word]['WP'] += 1 # 긍정단어과 같은 문장 내 단어일 때
                    break
            for ne in neg:
                if ne in sentence:
                    tmp[word]['WN'] += 1 # 부정단어와 같은 문장내 단어일 때
                    break
    return pos, neg, pd.DataFrame(tmp)

In [10]:
pos, neg, new_dict = make_sent_dict(df['comment'].values)
new_dict

100%|██████████| 15/15 [00:00<00:00, 38.93it/s]


,윤석열,친,이,노선,타면서,디시,에서,개빨림,일베,가,...,승전,문재인,아니었어,그랬,왠,아까,그러던데자,수해,우린,믿어요
W,2,6,151,1,1,5,25,1,14,65,...,1,1,1,1,1,1,1,1,1,2
WP,2,6,140,1,1,5,22,1,11,61,...,1,1,1,1,1,1,1,1,1,2
WN,2,6,132,1,1,5,17,1,11,54,...,1,1,1,1,1,1,1,1,1,2


In [11]:
def make_score_dict(d,p,n):
    N=sum(d.iloc[0,::])
    pos_cnt=sum(d.loc[::,p].iloc[0,::])
    neg_cnt=sum(d.loc[::,n].iloc[0,::])
    
    trans =d.T
    trans['neg_cnt']=neg_cnt
    trans['pos_cnt']=pos_cnt
    trans['N']=N

    trans['MI_P']=np.log2(trans['WP']*trans['N']/trans['W']*trans['pos_cnt'])
    trans['MI_N']=np.log2(trans['WN']*trans['N']/trans['W']*trans['neg_cnt'])
    trans['SO_MI']=trans['MI_P'] - trans['MI_N']
    
    trans = trans.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    trans = trans.sort_values(by=['SO_MI'],ascending=False)
    return trans

In [12]:
t_dict = make_score_dict(new_dict,pos,neg)
t_dict['SO_MI'] = scaler.fit_transform(t_dict['SO_MI'].values.reshape(-1,1))
t_dict

/Users/yongheon/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


,W,WP,WN,neg_cnt,pos_cnt,N,MI_P,MI_N,SO_MI
어제,4,4,1,50,72,4598,18.336716,15.810647,2.000000
뒤,3,3,1,50,72,4598,18.336716,16.225684,1.615877
기자,3,3,1,50,72,4598,18.336716,16.225684,1.615877
극우,3,3,1,50,72,4598,18.336716,16.225684,1.615877
엄청,3,3,1,50,72,4598,18.336716,16.225684,1.615877
...,...,...,...,...,...,...,...,...,...
사이버,2,1,2,50,72,4598,17.336716,17.810647,-0.776539
없음,3,1,3,50,72,4598,16.751753,17.810647,-1.317929
머,3,1,3,50,72,4598,16.751753,17.810647,-1.317929
뉴스,3,1,3,50,72,4598,16.751753,17.810647,-1.317929


In [13]:
def update_dict(d):
    add_Dic = {0:[],1:[]}
    for i in d.T.items():
        if i[0] not in list(sent_dic[0]):
            if len(i[0]) > 1:
                add_Dic[0].append(i[0])
                add_Dic[1].append(i[1]['SO_MI'])
            
    add_Dic=pd.DataFrame(add_Dic)
    Sentiment=pd.merge(sent_dic,add_Dic,'outer')
    return Sentiment

In [14]:
add_dict =update_dict(t_dict)
add_dict

,0,1
0,(-;,1.000000
1,(;_;),-1.000000
2,(^^),1.000000
3,(^-^),1.000000
4,(^^*,1.000000
...,...,...
15936,책임,-0.776539
15937,사이버,-0.776539
15938,없음,-1.317929
15939,뉴스,-1.317929


In [28]:
def get_cnt(x):
    cnt = 0
    for word in list(set(x)):
        target = add_dict[add_dict[0]==word]
        if len(target)==1:
            cnt += float(target[1])
    return cnt

def get_ratio(x):
    score = x['score']
    length = np.log10(len(x['comment']))+1
    try:
        ratio= round(score/length,2)
    except:
        ratio = 0
    return ratio

In [29]:
tqdm.pandas()
df['score']= df['comment'].progress_apply(lambda x : get_cnt(x))
df['ratio'] = df.apply(lambda x: get_ratio(x), axis = 1)

100%|██████████| 15/15 [00:03<00:00,  4.31it/s]


In [30]:
df

,sentence,comment,score,ratio
0,"윤석열 친이 노선 타면서 디시에서 개빨림,디시 일베가 윤석열만이 우리나라유일의 희...","[윤석열, 친, 이, 노선, 타면서, 디시, 에서, 개빨림, 디시, 일베, 가, 윤...",19.690427,5.82
1,"어제자 그알 보고 느낀 점,코로나가 염병이긴 염병이구나코로나로 온 동네 노인정 다...","[어제, 자, 그, 알, 보고, 느낀, 점, 코로나, 가, 염병, 이긴, 염병, 이...",31.248142,9.94
2,"그알이 남긴 역사로 남을 짤,,조선일보도 못믿는다더라 탄핵시국에 통수쳤다고 촌의 뒤...","[그, 알, 이, 남긴, 역사, 로, 남, 을, 짤, 조선일보, 도, 못, 믿는다더...",19.739617,6.19
3,"그알이 비겁한게 언레기는 안 다루더라,언레기 책임은 안 다루고 전부 유튜버로 몰아가...","[그, 알, 이, 비겁한게, 언레기, 는, 안, 다루더라, 언, 레기, 책임, 은,...",25.493791,6.98
4,"한국만 예외인 고학력자의 진보화, 저학력 저소득층을 지지기반으로 가지던 좌파정당이...","[한국, 만, 예외, 인, 고학력자, 의, 진보, 화, 저, 학력, 저, 소득, 층...",47.202971,12.13
5,"그알에서 뭐랬길래 인정하는 사람들이 생김,남유게인지 여기인지 모르겠는데 반응 보니깐...","[그, 알, 에서, 뭐랬길, 래, 인정, 하는, 사람, 들, 이, 생김, 남유, 게...",18.896808,5.52
6,"암만봐도 한강에 과몰입한건 딱 그 부모 정도 나이임,젊은층도 있겠지만 박종철 열사 ...","[암만, 봐도, 한강, 에, 과, 몰입, 한, 건, 딱, 그, 부모, 정도, 나이,...",31.515090,9.00
7,"한강 사건이 말해주는 대형렉카와 좋소렉카의 차이,좋소렉카 어떻게든 끌고 갈려고 함...","[한강, 사건, 이, 말, 해주는, 대형, 렉카, 와, 좋소, 렉카, 의, 차이, ...",6.683704,2.22
8,"한강사건 요약, 언론 믿지마 일베를믿어,지능이 딸리는것들이 유튜브 광신하더라대표적인...","[한강, 사건, 요약, 언론, 믿지마, 일베, 를, 믿어지, 능이, 딸리는것, 들이...",9.381214,2.81
9,"오늘자 그알에 나온 유튜브 채널 곳,밀쳐지는 손을 봤다는 분수술용 메스를 휘둘렀다는...","[오늘, 자, 그, 알, 에, 나온, 유튜브, 채널, 곳, 밀쳐지는, 손, 을, 봤...",23.466818,7.12
